In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw

In [3]:
import pandas as pd

In [4]:
from praw.models import MoreComments

In [5]:
import re

In [6]:
# Filling in the reddit webscraper authentication information
reddit = praw.Reddit(client_id='jAoa5j1cprR-ug', client_secret='rVHMYJMx53AmZWKYcgVK3WVyERALnA', user_agent='RateMe Scraping')

In [7]:
# looping and collecting information on the top 100 posts on RateMe
top_list = []
rm_subreddit = reddit.subreddit('RateMe')
for post in rm_subreddit.top(limit=1500, time_filter= 'all'):
    top_list.append([post.title, post.score, post.id, post.url, post.num_comments, post.created])
top_df = pd.DataFrame(top_list,columns=['title', 'score', 'id', 'url', 'num_comments', 'created'])

In [8]:
top_df

,title,score,id,url,num_comments,created
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09
...,...,...,...,...,...,...
986,[M20] Please be nice to me haha. I'm open to c...,105,kevv52,https://i.redd.it/vrx9vheehq561.jpg,62,1.608234e+09
987,[F22] Posted here 3 years ago; my look has cha...,101,k9i1w1,https://i.redd.it/utrg89l9f2461.jpg,126,1.607507e+09
988,"[24F] Can never make it to the ""exclusivity st...",89,k9c3wp,https://i.redd.it/b6o6l78lu0461.jpg,92,1.607488e+09
989,"[M26] just turned 26, drinking by myself on my...",83,kh7e90,https://i.redd.it/nkvcyjsi4g661.png,56,1.608544e+09


In [9]:
# adding the top 15 comments of each post as a series to top_df

def top_comment_extractor(df):
    top_comments = []
    for post_id in df['id']: 
        submission = reddit.submission(id = post_id)
        submission.comments.replace_more(limit=0) # needed for AttributeError
        top_comments.append(submission.comments[:15]) # getting top 10 comments of each post
    return top_comments
    

In [10]:
top_df["comments"] = top_comment_extractor(top_df)

In [11]:
# beauty_extractor take a string, and either return "False" if no score exists in such comment
# or the score if it does exist. 
# Score is in the range of [0,10]
# E.g of comments and function outputs
# You have really nice bone structure, my guy! 7/10 -> 7
# The 2 pics seriously don't look like the same person. -> False
# Face 7/10, Body 10/10 -> 8.5
# I'm put you at 7 for now. -> False (future fix)
# She was a 1/10 before, now she's a 3 or a 4 tops. -> 1 (future fix)

def beauty_extractor(comment):
    find = re.compile(r".+?(?=\/10)")
    times_scores_appear = 0
    total_scores_given = 0
    space_separated_comment = comment.split(" ")
    #print(space_separated_comment)
    for word in space_separated_comment:
        try: 
            rating_given = float(re.search(find, word).group(0))
            if ((rating_given > 10) or (rating_given < 0)) : continue # ratings are [0,10]
            total_scores_given += rating_given
            times_scores_appear += 1
        except: 
            continue # if it is not a "x/10" string
    #print(times_scores_appear)
    if (total_scores_given == 0) : return False
    else: return(total_scores_given / times_scores_appear)

In [12]:
top_df

,title,score,id,url,num_comments,created,comments
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ..."
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ..."
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ..."
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ..."
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ..."
...,...,...,...,...,...,...,...
986,[M20] Please be nice to me haha. I'm open to c...,105,kevv52,https://i.redd.it/vrx9vheehq561.jpg,62,1.608234e+09,"[gg4sibe, gg5ux1x, gg5vfgw, gg5y54q, gg5w7ua, ..."
987,[F22] Posted here 3 years ago; my look has cha...,101,k9i1w1,https://i.redd.it/utrg89l9f2461.jpg,126,1.607507e+09,"[gf4dqy2, gf5pvze, gf4vn76, gf4wweh, gf532ud, ..."
988,"[24F] Can never make it to the ""exclusivity st...",89,k9c3wp,https://i.redd.it/b6o6l78lu0461.jpg,92,1.607488e+09,"[gf38cti, gf5c2ue, gf3qifg, gf3rb49, gf3y7hs, ..."
989,"[M26] just turned 26, drinking by myself on my...",83,kh7e90,https://i.redd.it/nkvcyjsi4g661.png,56,1.608544e+09,"[ggjk6fy, ggjtnkx, ggk3uye, ggjrjnp, ggjwkjp, ..."


In [13]:
# finds the average rating for a user, for all the users. Rating is return in a list.
# Rating is calculated by:
# (Rating out of 10 * Upvotes of that comment) / (All upvotes on comments with ratings on post)
#E.g:
# Face 7/10, Body 10/10, 70 upvotes
# 6/10, 39 upvotes
# Total score: 8.5 * 70 + 6 * 39 / (70 + 39) = 7.61 (2SF)

def average_rating_extractor(df):
    average_ratings = []
    for individual_post_comments in df['comments']: # iterating through all posts
        all_ratings = 0
        all_scores = 0
        for one_comment in individual_post_comments: # top 10 comments of each post
            print(one_comment.body)
            if (beauty_extractor(one_comment.body) != False): #if comment gave a rating
                comment_score = one_comment.score
                all_ratings += (beauty_extractor(one_comment.body) * comment_score)
                all_scores += comment_score
        if (all_scores != 0): average_ratings.append(all_ratings / all_scores)
        else: average_ratings.append('Not Available')
    return average_ratings
    

In [14]:

top_df["average_rating"] = average_rating_extractor(top_df)

I don’t know if you want constructive criticism on the way you look but I was scrolling through this page and your smile caught my eye! You are definitely radiating happiness which to me is most important. I’m a girl who’s struggling with eating and body image issues and I hope to be this happy with myself one day :)!!
[deleted]
[deleted]
Very pretty and I love your hair. You remind me of Bryce Dallas Howard.
8.5/10. Look like Ariel from Little Mermaid
Your smile is super radiant and you literally look like you should be working at Disney. Your hair looks like a princesses. And the Weight goes in all the right places you’re not a hanger babe you’re a woman.
These people
Your smile is really pretty and you radiate happiness, which is very attractive. Solid 7.5 out of ten.
Your smile says everything; there is a glow to your presence that definitely adds to your prettiness. I believe that life is too short to deprive ourselves of things we enjoy completely. Everything in moderation :)
[de

7/10 you look very cute,you remind me of johanna kutchka bc of your aesthetic
8/10! You look like an elf 😍🤩
Very plain faced however nice figure and kind aura. Probably 6.5 on a bad day 8 on the best 
Pretty Lass
5.5  / 10

You are cute
6/10. The right make up could bump you up 1 point. You have lovely, fine features.
You're cute. 6/10
7.5/10. This is subjective but I personally think a different hairstyle would suit you better. Regardless you have an attractive face.
Cute af, 8.
You’re really pretty, I don’t think you need any make up, but a little wouldn’t hurt, I wouldn’t use  a lot though if I was you, maybe some eye liner and light lippie.
6/10
[deleted]
Very cute. 7ish. Reminds me of somebody from like, Little House On The Prairie or Little Women. You know, one of those time piece films. I don't know exactly what that means but there you go.
6
6.5/10
7/10
You can’t really do runway or catwalk modeling most women who are models at that height are picture/print models.

Fashion or 

[deleted]
8.5/10, can't really beat a pretty face and nice ass combo. When you hit your 20s, you seriously could be a 10.
You look like a younger version of  Rosie Huntington-Whiteley.  You're very beautiful.  8.5/10
I've seen worse
Gawdamn! You probably already know your rating.
Cute and young with potential to be hot in your 20s.
aye if she 18, im 18
Not gonna lie, I think you're getting overrated because of the photo quality and the blonde hair/blue eyes thing. I think you're a cute 6.5/7.
You know you're hot. Get outta here.
As an American who spent the spring semester in Scandinavia, you are way above average even there. In america you'd still be minimum an 8
Ayyy okay
[deleted]
Come on girl, you're a bombshell of a woman, enjoy your beauty and stop thinking you're not good looking.
You have a stunning face! You are truly beautiful
Monkey on the shoulder, and parrot on the foot.

You are the Disney Princess of the beach apparently.
Love the face!

7.5/10
Am I the only one who thin

When you're serious you're a 5.5/10. But when you smile you're a 7/10.
7 / 10

You are pretty
Quite attractive 7.50/10
You seem nice but something feels off for me. Can't put my finger on it. 7
24/93. 

You have a lot of years left ahead of you, enjoy them.
6/10, you look more pretty when you smile..
Solid 8. Easily.
Can't go any lesser then 7.5
8/10 you're cute
The pouting doesn’t suit you as much as the smile!
8.5 /10 love the smile❤️
6/10 - Upper Average

The majority here have pointed out your smile which does make you look more attractive. You have good fashion sense and long dark hair which is attractive also.

You do look slightly young for your age however this isn't a bad thing at all.
6
6/10
You look like you want to feed me hot wings and ask me questions. In other words, looking good!
90 degrees to the right and you’ll look a lot better
Looked at the before picture, you look waaaaaaay better with the shaved head.
You look great. You also have a "clean" appearance, as if you 

9/10
8.5/10 for me. You have a very pretty face imo
6
Good
6.5/10
Solid 7
5/10
6. Nice body, though.
6/10
5.5 But maybe a 6 with good clothes and makeup
[6/10](https://i.imgur.com/UsKHIsF.png)
5/10. You’re a cutie pie
6ish, I think. Cute and seems like you have a nice personality
[deleted]
6.5 / 10

You are cute
[deleted]
[deleted]
9/10 I don’t know if I’m biased or not lol
[deleted]
At least an 8. You're very pretty, with a wonderfully expressive face and an awesome looking body
8.4/10.  I looked through your post history and saw your before picture, and I have to say this.  What you have done is what so many people try but fail to do, and you should be so proud of yourself.  The way you have turned around your life is inspiring.  
Mostly because you represent my taste/style...9/10. For sure a 7/10 outside of the style 
7 or 8 depending on the pic. You're a doppelgänger for Ellen Page, and I can't believe nobody else mentioned that so far. Your hair's absolutely perfect and you have a

...

I'd give you a 6 or 7/10
Very pretty face.

I want to see what your body looks like though. If you're feeling insecure with a face like that, then odds are that your body isn't great.
7/10 because I don't know how your body looks but you have nice eyes
7/10
You're quite pretty. Great eyes. Hate those glasses. No body pics, so I think 6.5/10
~8
You're pretty, but wearing too much makeup for my taste. I like your cat though.
Nice glasses.
You are extremely pretty. Your face is so cute and your eyes are fantastic. 

You should wear those glasses more often. 
I don't like giving out number and objectifying women (LMAO).  Anyway, you're adorable and I'd gamble that your personality is great as well. 
Looks like you have more make-up on the left side of your face than on your right.  7/10.
7
5/10
You look older than 20 judging by your clothes, they kinda look like something a nurse would wear. But you are really pretty. I think a fashion change and just basic makeup shade on your face will m

7 having a degree helps for sure
7. I am not sure about your hair from the front. You look nice
5/10. Haircut could be better and would def improve your look
If being 6’5” isn’t enough of a cheat code.. trim your eyebrows a lil bit and get a fresh cut, short on the sides and combover on the top. 10/10 my guy
On your feet bro
Everyone has an asymmetrical face if you look hard enough. I didn't notice anything about your jaw; you're good
6/10, trim those eyebrow a bit..
9/10 I never actually noticed your jaw, I think you're really pretty and you have such a wonderful and warm smile with really captivating eyes! I like your style also, it's very trendy :)
Yeaaaa problem is your eyebrows they really drain out everything else good thing is that’s an easy fix
Supppppper pretty! Nothing wrong with your jaw, defo not noticeable to others! You do look young, I think you should totally try growing out your hair it would suit you perfectly and make you look a bit older plus I defo think the best t

maybe I’ll straighten my teeth one day if I can ever become less of a pussy ass bitch and actually go to the dentist 🙃
5.5 / 10

You are cute
5/10
6/10
5.7
5/10 but could b a 8 if u change some stuff around
6.5
Hi there, thank you for your submission! To be rated on /r/Rateme, a verification picture is required as stated in Rule 8. **One photo should include a HANDMADE sign that says /r/RateMe, and your EXACT Reddit username. Your face must be identifiable in the picture. If you have not added a verification picture, please make a new post that includes one as your current post will not appear.** All posts are manually approved by a moderator. Moderators reserve the right to question the authenticity of any submission. 

If your current post **does** include your verification and follows the 7 other rules on the sidebar, sit back and relax! It will appear after a little while. If your submission does not appear within 16 hours, please message the moderator team. Thank you for your pati

I like both versions but the right ones are definitely more conventionally attractive 8/10
I think the short hair suits you, but I definitely think that you pull of longer hair in the front (third pic down on the left side looks really nice!) and yeah I agree with the other comments I think that the new style definitely makes you look more mature but I honestly do really love the earrings as well!!
Reddit's given u bad advice, you looked  better before
I actually like the left more it looks more masculine. I liked the hair and earring it gave you personality. Your eyes looked more alive and deep on the left as well.
Way better now. 9/10
Easily a good 9/10. You looked great before and still look great. The shorter hair definitely made you look more mature tho. I appreciate your physique too, I can see how much work you’ve put in!
Hot I already feel attracted 9/10
I think you're pretty handsome no matter the rights or lefts pics, and I really think the best haircut is the one you like to

I think you look amazing. Maybe think about getting a tan *shrug*
Love the retro look, totally my type. I'd say 8.75/10.
Pretty, but Too much make up for my personal taste
You have an old school Hollywood beauty about you, and a fit body to boot. I dont know what youre insecure about.

8/10
[deleted]
9 out of 10. One of the best ratings I've given. Beautiful eyes, pretty face,  and damn I like that ass. 

Just my two cents, look for a better guy and not worry about your looks. 
You look very dramatic to me. I would stay away, although I do think you are attractive
Frank'n'Furter/10
7/10
An 8 for me, what puts you down from 9 or 10 is because you don't like happy and it comes across that you are pretty sad/unconfident in the photos by your demeanour.

I know what it's like to feel like that and I have days where I feel similar, it sucks.
honestly you look the most like someone id want to get to know when you're just rocking the hoodie. That being said I do love youre style, its definite

In [15]:
## Extracting gender out of each submission
def age_extractor(df):
    ages = []
    for title in df['title']:
        print(title)
        try:
            ages.append(re.findall('\d+', title)[0])
        except:
            ages.append("Not Available") # if there is no age
    return ages

In [16]:
top_df['age'] = age_extractor(top_df)

[26F] Hi Reddit :) Since I last posted, I stopped fussing over eating healthy and started eating what I wanted. I may have gained weight but I have never felt more beautiful
[F21] I lost 100lbs in the last year, I'm curious where I stand now!
[M60] Retired but not out, giving a last go to my lifelong dream of becoming a model, do I have it in me?
[F18] ex-goth who dropped out of college, tear my self esteem to shreds!
(F21) just curious.
Hello... I have seen your post.. I am a person with common intrest. Never lie, emotional and sesitive, honest by heart, downto earth, cool and calm nature. Hiii... I want good, true and honest friendship from all over the world.. I am 40/m/India, i am deeply spiritual person... 🙏
Hello again Reddit! I’m still getting requests for a full body shot and I finally worked up the courage to do it. You don’t need to rate my cat though, I already know she’s perfect
[31M] Recovering anorexic. Halfway there. Started at 90 lbs and now at 130 lbs (I'm 5'11"). Gave

In [17]:
top_df

,title,score,id,url,num_comments,created,comments,average_rating,age
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ...",8.36667,26
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ...",8.64939,21
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ...",10,60
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ...",7.53097,18
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ...",8.24214,21
...,...,...,...,...,...,...,...,...,...
986,[M20] Please be nice to me haha. I'm open to c...,105,kevv52,https://i.redd.it/vrx9vheehq561.jpg,62,1.608234e+09,"[gg4sibe, gg5ux1x, gg5vfgw, gg5y54q, gg5w7ua, ...",8.04348,20
987,[F22] Posted here 3 years ago; my look has cha...,101,k9i1w1,https://i.redd.it/utrg89l9f2461.jpg,126,1.607507e+09,"[gf4dqy2, gf5pvze, gf4vn76, gf4wweh, gf532ud, ...",7.27778,22
988,"[24F] Can never make it to the ""exclusivity st...",89,k9c3wp,https://i.redd.it/b6o6l78lu0461.jpg,92,1.607488e+09,"[gf38cti, gf5c2ue, gf3qifg, gf3rb49, gf3y7hs, ...",8.5,24
989,"[M26] just turned 26, drinking by myself on my...",83,kh7e90,https://i.redd.it/nkvcyjsi4g661.png,56,1.608544e+09,"[ggjk6fy, ggjtnkx, ggk3uye, ggjrjnp, ggjwkjp, ...",3.28125,26


In [18]:
# given a string with a gender containing a gender written as 
# "[20f], 20female, (20f), 20/f/dubai", parses the age
def gender_parser(title):
    
    # matches 24male
    if (len(re.findall(r'(\d+)\s*(male|female)', title, re.IGNORECASE)) != 0):
        return re.findall(r'(\d+)\s*(male|female)', title, re.IGNORECASE)[0][1][0].upper() # first letter of gender
    
    #matches 23f
    if (len(re.findall(r'(\d+)\s*(m|f)', title, re.IGNORECASE)) != 0):
        return re.findall(r'(\d+)\s*(m|f)', title, re.IGNORECASE)[0][1][0].upper()
    
    # matches f23
    if (len(re.findall(r'(m|f)\s*(\d+)', title, re.IGNORECASE)) != 0):
        print(title)
        return re.findall(r'(m|f)\s*(\d+)', title, re.IGNORECASE)[0][0][0].upper()
    
    # matches 24/m
    for word in title.split('/'): 
        if (word in 'mMfF'):
            return word.upper()
        
    return "Not Available"

In [19]:
# given a df with a title category, returns a list of all ages
def gender_extractor(df):
    gender_list = []
    for title in df['title']:
        gender_list.append(gender_parser(title))
    return gender_list

In [20]:
top_df.head(10)

,title,score,id,url,num_comments,created,comments,average_rating,age
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ...",8.36667,26
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ...",8.64939,21
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ...",10,60
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ...",7.53097,18
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ...",8.24214,21
5,Hello... I have seen your post.. I am a person...,1409,e4c8kv,https://i.redd.it/669rnmwoty141.jpg,241,1.575210e+09,"[f9ae009, f993z64, f98vvyp, f9av7jp, f999fhb, ...",7.66667,40
6,Hello again Reddit! I’m still getting requests...,1116,9ytcg2,https://i.redd.it/2qcuxq9bciz11.jpg,207,1.542758e+09,"[ea3w0xl, ea46oyx, ea3vqo5, ea41o06, ea40gwr, ...",8.456,Not Available
7,[31M] Recovering anorexic. Halfway there. Star...,1106,93hajs,https://imgur.com/a/KyLLVRn,122,1.533092e+09,"[e3ddnz0, e3ded40, e3dejt1, e3disgw, e3dmbiz, ...",6.9625,31
8,I posted on this subreddit a while back when m...,1091,cavulm,https://i.redd.it/hjrbtgzxe7931.jpg,130,1.562675e+09,"[etbjb7m, etbh463, etbh5jf, etbhbis, etbivm6, ...",8.42553,22
9,30[F]. My boyfriend left me for a 24 year old ...,1079,243rja,http://imgur.com/WqYBMT2,276,1.398637e+09,"[ch3c6zj, ch3gi2w, ch3b4mi, ch3b85c, ch3chso, ...",8.38462,30


In [21]:
top_df['gender'] = gender_extractor(top_df)

[F21] I lost 100lbs in the last year, I'm curious where I stand now!
[M60] Retired but not out, giving a last go to my lifelong dream of becoming a model, do I have it in me?
[F18] ex-goth who dropped out of college, tear my self esteem to shreds!
(F21) just curious.
Hello... I have seen your post.. I am a person with common intrest. Never lie, emotional and sesitive, honest by heart, downto earth, cool and calm nature. Hiii... I want good, true and honest friendship from all over the world.. I am 40/m/India, i am deeply spiritual person... 🙏
[F18] just curious :)
[f19] Honest rate? I’ll rate people back if they want me to :)
Hello Reddit, it’s nice to meet you [F25]
[F24] Trying hard to learn to show teeth when smiling
Had a small makeover since last year and I would love to hear your opinion [M22]
Sorry I couldnt get the framing too great, but opinions? Improvements? F20, 110lbs, 5'6". (More pics on my profile too)
F20 Quarantined and willing to get rated.
(F18) It's my first time po

In [22]:
top_df

,title,score,id,url,num_comments,created,comments,average_rating,age,gender
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ...",8.36667,26,F
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ...",8.64939,21,F
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ...",10,60,M
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ...",7.53097,18,F
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ...",8.24214,21,F
...,...,...,...,...,...,...,...,...,...,...
986,[M20] Please be nice to me haha. I'm open to c...,105,kevv52,https://i.redd.it/vrx9vheehq561.jpg,62,1.608234e+09,"[gg4sibe, gg5ux1x, gg5vfgw, gg5y54q, gg5w7ua, ...",8.04348,20,M
987,[F22] Posted here 3 years ago; my look has cha...,101,k9i1w1,https://i.redd.it/utrg89l9f2461.jpg,126,1.607507e+09,"[gf4dqy2, gf5pvze, gf4vn76, gf4wweh, gf532ud, ...",7.27778,22,F
988,"[24F] Can never make it to the ""exclusivity st...",89,k9c3wp,https://i.redd.it/b6o6l78lu0461.jpg,92,1.607488e+09,"[gf38cti, gf5c2ue, gf3qifg, gf3rb49, gf3y7hs, ...",8.5,24,F
989,"[M26] just turned 26, drinking by myself on my...",83,kh7e90,https://i.redd.it/nkvcyjsi4g661.png,56,1.608544e+09,"[ggjk6fy, ggjtnkx, ggk3uye, ggjrjnp, ggjwkjp, ...",3.28125,26,M


In [23]:
# Removing all rows without either Age or Gender data
top_df = top_df[top_df['age'] != 'Not Available']
top_df = top_df[top_df['gender'] != 'Not Available']
top_df = top_df[top_df['average_rating'] != 'Not Available']

In [24]:
top_df

,title,score,id,url,num_comments,created,comments,average_rating,age,gender
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ...",8.36667,26,F
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ...",8.64939,21,F
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ...",10,60,M
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ...",7.53097,18,F
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ...",8.24214,21,F
...,...,...,...,...,...,...,...,...,...,...
986,[M20] Please be nice to me haha. I'm open to c...,105,kevv52,https://i.redd.it/vrx9vheehq561.jpg,62,1.608234e+09,"[gg4sibe, gg5ux1x, gg5vfgw, gg5y54q, gg5w7ua, ...",8.04348,20,M
987,[F22] Posted here 3 years ago; my look has cha...,101,k9i1w1,https://i.redd.it/utrg89l9f2461.jpg,126,1.607507e+09,"[gf4dqy2, gf5pvze, gf4vn76, gf4wweh, gf532ud, ...",7.27778,22,F
988,"[24F] Can never make it to the ""exclusivity st...",89,k9c3wp,https://i.redd.it/b6o6l78lu0461.jpg,92,1.607488e+09,"[gf38cti, gf5c2ue, gf3qifg, gf3rb49, gf3y7hs, ...",8.5,24,F
989,"[M26] just turned 26, drinking by myself on my...",83,kh7e90,https://i.redd.it/nkvcyjsi4g661.png,56,1.608544e+09,"[ggjk6fy, ggjtnkx, ggk3uye, ggjrjnp, ggjwkjp, ...",3.28125,26,M


In [25]:
top_df.head(10)

,title,score,id,url,num_comments,created,comments,average_rating,age,gender
0,"[26F] Hi Reddit :) Since I last posted, I stop...",1942,bzh80r,https://i.redd.it/erdysfj57s331.jpg,244,1.560311e+09,"[eqsn2l6, eqspwxq, eqsv29p, eqstem6, eqt1oal, ...",8.36667,26,F
1,"[F21] I lost 100lbs in the last year, I'm curi...",1792,fz6beu,https://i.redd.it/miwsxnty77s41.jpg,271,1.586644e+09,"[fn3qvn8, fn39lp7, fn3c9sy, fn3gksl, fn3ivtr, ...",8.64939,21,F
2,"[M60] Retired but not out, giving a last go to...",1726,3sgqzw,http://imgur.com/a/63Zrl,142,1.447311e+09,"[cwx89kv, cwx53la, cwx3zj4, cwx2kmh, cwxhaal, ...",10,60,M
3,"[F18] ex-goth who dropped out of college, tear...",1599,b8mjff,https://i.redd.it/6covsqh73wp21.jpg,322,1.554256e+09,"[ejys2vq, ejys8be, ejyt6ym, ejys5uh, ejyzn0w, ...",7.53097,18,F
4,(F21) just curious.,1506,edfbsl,https://i.redd.it/sc4o7j5diu541.jpg,291,1.576901e+09,"[fbi91dz, fbidtnw, fbip0qy, fbja0o4, fbi7s5h, ...",8.24214,21,F
5,Hello... I have seen your post.. I am a person...,1409,e4c8kv,https://i.redd.it/669rnmwoty141.jpg,241,1.575210e+09,"[f9ae009, f993z64, f98vvyp, f9av7jp, f999fhb, ...",7.66667,40,M
7,[31M] Recovering anorexic. Halfway there. Star...,1106,93hajs,https://imgur.com/a/KyLLVRn,122,1.533092e+09,"[e3ddnz0, e3ded40, e3dejt1, e3disgw, e3dmbiz, ...",6.9625,31,M
8,I posted on this subreddit a while back when m...,1091,cavulm,https://i.redd.it/hjrbtgzxe7931.jpg,130,1.562675e+09,"[etbjb7m, etbh463, etbh5jf, etbhbis, etbivm6, ...",8.42553,22,M
10,[20f] I chopped my hair off,1021,cr9vfz,https://i.redd.it/h4cy63zylug31.jpg,131,1.566006e+09,"[ex3lz09, ex3xz93, ex3qiqi, ex3s775, ex41lhz, ...",8.45492,20,F
11,"[30F] turning 31 tomorrow, rate me!",1012,aa0x5c,https://i.redd.it/w9rpgcrkqu621.jpg,392,1.545959e+09,"[eco37pl, ecogmp9, ecowoeg, eco7yhr, eco84mh, ...",7.37327,30,F


In [26]:
top_df[["age"]] = top_df[["age"]].apply(pd.to_numeric)
top_df[["average_rating"]] = top_df[["average_rating"]].apply(pd.to_numeric)
top_df[["score"]] = top_df[["score"]].apply(pd.to_numeric)

In [27]:
# What's the corelation between attraction and score?
top_df['average_rating'].corr(top_df['score'])

0.14918105108806964

In [28]:
# How many males are in the top vs females?
top_df[top_df["gender"] == "M"].shape[0]
# 177 / 854 

178

# Doing the same thing but with controversial posts

In [29]:
# looping and collecting information on the top 100 posts on RateMe
controversial_list = []
rm_subreddit = reddit.subreddit('RateMe')
for post in rm_subreddit.controversial(limit=1500, time_filter= 'all'):
    controversial_list.append([post.title, post.score, post.id, post.url, post.num_comments, post.created])
controversial_df = pd.DataFrame(controversial_list,columns=['title', 'score', 'id', 'url', 'num_comments', 'created'])

In [30]:
controversial_df

,title,score,id,url,num_comments,created
0,Rate me honestly please (f) 18,4,1sl57q,http://imgur.com/a/fp5PW,31,1.386746e+09
1,"[18m] amab but genderfluid, be honest i know i...",17,jreqp3,https://i.redd.it/6vnhrm05hcy51.jpg,67,1.605013e+09
2,(M)21 How do I look?,0,11uznc,http://i.imgur.com/xzUPd.jpg,4,1.350882e+09
3,[18F] :),1,2uu9x6,http://imgur.com/a/bb0kl,42,1.423141e+09
4,[F19] Haven't had the greatest luck with men a...,129,946bd3,https://imgur.com/a/cmhUqbZ,277,1.533297e+09
...,...,...,...,...,...,...
988,25F re-upload with the username not backward,0,kih9kj,https://imgur.com/a/F40uhIe,16,1.608710e+09
989,[22f] I need honestly feedback & rates!!,16,j2urm9,https://i.redd.it/du9wbw5iocq51.png,31,1.601529e+09
990,27f new mom. 5'11 170lbs. Rate me.,2,kqgg5h,http://imgur.com/a/mVVJWCN,4,1.609819e+09
991,"M25, would like to know what you think especia...",123,kjm1ua,https://i.redd.it/78f9dse217761.jpg,82,1.608870e+09


In [31]:
controversial_df["comments"] = top_comment_extractor(controversial_df)
controversial_df["average_rating"] = average_rating_extractor(controversial_df)
controversial_df['age'] = age_extractor(controversial_df)
controversial_df['gender'] = gender_extractor(controversial_df)

The fact that you are also posting on /r/seduction ([here](http://www.reddit.com/r/seduction/comments/1lmovr/do_i_have_to_change_my_personality_normally_very/)) makes me think this probably isn't you.
Grats! You made it to /r/quityourbullshit !
3/10
9.
Very pretty.  Cute.  Nice freckles.  Excellent feet. I think you're easily a 7, maybe 7.5ish, and once you get the braces off it'll be even more apparent.
once your braces come off you'll look a lot better, 7 when they come off 6 now
Aww, you're very cute! I love your freckles! You've got a wonderful smile, I love how pronounced the apples of your cheeks are! If I were you I'd play them up with a little bit of nice natural blush. Your eyebrows are great, if you filled them in with a little pencil they'd really add some extra oomph to your face. That's all the advice I can give you! You're a naturally beautiful girl with lots of great features. I think my favorite pic is the second to last one when your hair is parted to the side and I ca

I love you.
I don't dig the steampunk style and don't buy it that you can't see how you look feminine, but other than the clothes I think you look pretty badass. More punk, less steam  and I would be simultaneously intimidated and enchanted by you. Steam punk stuff just makes it hard for me to take you seriously. 
Wow you have the most proportional face i have ever seen...your style is edgy and unique but not my cup of tea.if you were naked with no piercings and no makeup i would rate you a 10/10 a perfect artist's muse. 
cool hat and goggles
I see nothing wrong here
0/10.  Dead link.

Zoinks! You've taken a wrong turn.
Let's split up, gang. If you're looking for an image, it's probably been deleted or may not have existed at all.
[deleted]
8/10 - Very cute, got my heart rate nice and elevated :)
I'm 23M.  I think you're too skinny.  You're cute but building some muscle and even a little fat would make you hot!
Definitely 8.5/10, attractive. 
You're a bit on the too slim side IMO. 6.5/

2.../10
3
Drink less and lose weight, you will be above average if you do.
looking sloppy...could be the drunkeness...3.6/10
..oh god
Being the kind of person that has pictures of herself when she's sober as well *may* make you into a more appealing person overall.
3.14/10
20? You look 28
I think youre fucking adorable. 
loose weight 4/10
5
cute.. nice hair, great smile
please never dye your hair.  looking cute. 
You look Shrek-tastic.
You look pretty, but I would never talk to you. I don't think I'd like you when you're angry.
I honestly think that you should post another photo which could allow us to give you a fair rating.
Bad photo, but you seem to have a nice face shape and could be rather attractive
I gotta agree, green pictures...can't rate very well.
Tough picture to judge. It's usually good throwing multiple photos because I really don't know your weight or height. 

You have all the typical attributes for an attractive face though; You have the great eyes, lips, and overall f

I for one think you are lovely!  Your teeth are very nice, and your eyes are entrancing.  8.8/10?  Also, don't know how your hair sweeps upward when you are lying upside down... Sorcery?
Wow, I did not notice that literally every comment here is negative... Harsh.  You let me down, reddit community.
I dont think the short hair goes well with you, the filtering and make up also doesnt look great.  I don't think I would pin you as likely to be a particularly successful model
You look like Gill from Archer in that last pic. 
Your submission has been automatically removed because the title does not include one of the required tags. Please check the sidebar for information about tagging your submission properly

*[I am a bot](/r/AutoModerator/comments/q11pu/what_is_automoderator/), and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose?to=%2Fr%2FRateme) if you have any questions or concerns.*
I love the pink hair! I think your eyebrow

more pics?
5.5/10. Cute.
6.5
mmmbop
6
Well, you certainly are upside down.
You're definitely cute, but your photography isn't great!
Dizzy/10
'just turned 18' is usually followed by other pics... oh wait, that's the internet breaking me.
very cute.  8/10
5.
One upside down crappy quality photo?  4/10
I like that you look good without makeup

8/10

edit: would smash
I am so glad that most of my terrible mistakes lead to shit like this. 

You are so goddamn beautiful. 

Solid motherfucking ten.
[deleted]  
 ^^^^^^^^^^^^^^^^0.9651 
 > [What is this?](https://pastebin.com/64GuVi2F/12864)
hahahaha this picture is top notch.
LOL
I came
lol I like how you took a picture of some old pic
[Patrick Carney?](http://i.imgur.com/ZDzJ31y.jpg)
Haircuts, people. Haircuts for everyone.
You look a lot like someone I used to date.  Its kind of freaking me out.
Nine.
damn, super cute! 9/10 ;)
You're probably going to age into highly-desirable tech hipster in your 20s. But like all highly-desirable tech hip

...

You're killing the red lipstick. Love it. 
5/10, the 2nd and last pics are nice so I would replicate that look. Overall not bad looking, just very average.

As a sidenote, I'm just curious- what's the incentive in putting "asian" in the header? I'm an asian girl myself, but it still puzzles me because I've rarely seen it done for other races.
The last pic I would take notice of you. The others I probably wouldn't
40/100
Your Facial features look better with glasses compared to last pic I would just suggest diffrent frames 

6-7/10
6-7 in the last pic, but rather standard in the previous ones.

You look best without glasses (atleast for me) so maybe get contacts / wear them more often if you already have them?
Totally cute, but I also suspect that you're not very photogenic. Not an insult. Some people are stunning when you meet them, but don't photograph very well. I also find most photogenic people are underwhelming in person. 

I'm kind of getting a good feel for how you look in perso

6/10, but could be higher.  I think you could be overweight though.
Looks like has sass, means freaky. Yes. 
[deleted]
You've posted here before. Haven't you got the answer you're looking for by now? And Moto G (snap) ftw! 
multi pass?
Cute, 6.
very pretty and you'll be in shape your whole life.

7/10 all day every day
I like what I see/10
When you say "I'm an adult", I read that in Bernadette's voice (From the big bang theory)

Okay so, let's get the obvious out of the way, you do look quite young.

Putting that aside, you clearly pay attention to yourself, your make up and your clothing, so that's nice. 

Cute smile in photo 2. Definitely do more smiling.

Hair is alright. Not sure if the colour/cut suits you best but yeah it's not bad.

All in all probably 7/10

As in, I'd probably say hi. 
Adorbs! 8/10.
Wow... MY BRAND!!   I love little petite girls. 
I think your face is breath taking.  I like your hair.  I think you remind me a lot of Chloe Dykstra.  Which is good because I'm mad

Pretty good. I dig the beard :-D 7/10
Lol. So I'm not the only one to take shitter selfies!
You look like a Mexican George Michael, you know, from WHAM!
Ur a fag
I like the first pic best, cute smile, nice eyes.


Too much sclera showing under the eye paired with bulgy looking prominent eyes gives a slight strange look. 


Lips protrude too far out, makes chin look weak.
Facial structure too wide. 


Other than that, your skin is good, features symmetrical. 
Altogether I'd give around a 3, low 4. You can look at my comment history, I'm trying to say it like I see it. I don't want to bring someone down but you've asked, and I'm explaining honestly what I think of you. 
3.5.
3/10 for me.
4.5/10
2.5/10
well i'm sure you know about the obvious problem, and actually cant help but saying it takes you down to a 5/10, but if you manage to clear your skin you could easily be a 6/10, if not more  

10/10 - not achievable, would be overall perfect.  
9/10 - absolutely attractive, in all aspects  

cool shirt i guess, everything else is to dark to tell.
Is this a rating on the portrait or on your looks?

For the photo I'd mute the black a bit so they weren't so black. Love the contrast and lighting. Also patch out the neckline and bring out the eyes more with some local adjusting.

For the style of clothing, I'd get rid of the yellow belt thing, and the hat. I'd say don't wear the bracelet, but you are wearing a Bob Marley shirt, so keep it. ;) 
Great picture. I think you're handsome but it's hard to tell because of the shadows.

I think you definitely need some /r/malefashionadvice.
Tall, dark and handsome, huh?

You're VERY pretty, but you need a new wardrobe. Classy goes MUCH better for you then what you're currently wearing.

8, 8.5 properly dressed, maybe 8.7 if you've got a nice smile ;)
I think it looks awesome!
8+/10. 
You look a bit like a stoner. But a *cute* stoner, so 7-8/10 for now.
That creepy, from-the-bottom-up-ghost-stories-around-the-campfire lighting doesn't he

[F18] Boyfriend thinks my legs are hot but i'm not sure...Rate My Legs!
[F28] - Thoughts and comments?
[F18] trans, know I’m not anywhere near passing yet but plz tell me where I stand guys
F18 do it u won’t
[M22]2016 Rate Me
Hello! (F23) rate me idk how people perceive me.
M29 rate me
[M20] rate my body, I already know I have the face of a 12 year old. My girlfriend says I'm muscular but I still see the myself as the scrawny 13y/o I was in middle school
Rate Me Please (F23)
[F16] Rate me :)
[M21] Romanian Honest opinions 1/10. PD: The verification photo is me literally pulling an allnighter so it's very bad.
[M18] Tried a new haircut and my friend said i look like willy wonka and look like a girl, the second pic was you its so dark here rn sorry
(M19) Give me a rating
[m28] enby 1-10 pls!?
[M23] Thought I'd join in the fun
[M25] Rate me, please :-)
[M18] I am still learning how to adult. Tell me what you think!
[M21] Just got brutally dumped. Confidence in the shitter. Am I beyond sav

In [32]:
controversial_df['gender'] = gender_extractor(controversial_df)

[F19] Haven't had the greatest luck with men and am starting to feel insecure. How do I stand?
[F18] Let's try this again guys
[F - nearly 32] Single mom of 1 - Really nervous doing this...what do you think?
[F19] - Rate me?
F37 (yes 37, yes taken this year 2013, YES a picture of me) - just curious
Rate me [M29] cheers!!!
F 18 rate me reddit :) sorry I couldn't find a decent body shot
[M19] Have gotten a lot of great comments on reddit but still not feeling okay with looks at times.
Summer is here [F20]
[F18] Geek Girl Looking for a rating
Rate the Paramedics face [M18]
[F25] I'm fat and tattooed, wreck me
[F24] Just curious to see what people think of me
I actually don't want to be rated, but almost everyone tells me in real life that I look like Ed Sheeran [m27]
[M28] Rate me on my first six. Should I change them? Be brutal, thank you!
F28} wake up with massive jet lag in hotel room
It's been a while. We are back. [M22] NSFW
19 years young, (m)'ladies please rate me 1 out of 10
(M 18

In [33]:
# Removing all rows without either Age or Gender data
controversial_df = controversial_df[controversial_df['age'] != 'Not Available']
controversial_df = controversial_df[controversial_df['gender'] != 'Not Available']
controversial_df = controversial_df[controversial_df['average_rating'] != 'Not Available']

In [34]:
controversial_df

,title,score,id,url,num_comments,created,comments,average_rating,age,gender
1,"[18m] amab but genderfluid, be honest i know i...",17,jreqp3,https://i.redd.it/6vnhrm05hcy51.jpg,67,1.605013e+09,"[gbssthb, gbtc9qb, gbuah4k, gbv0tdn, gbv2glc, ...",5.13889,18,M
3,[18F] :),1,2uu9x6,http://imgur.com/a/bb0kl,42,1.423141e+09,"[cobqz1a, cobrpqb, cobrg8z, cobrseo, cobxur2, ...",8.75,18,F
4,[F19] Haven't had the greatest luck with men a...,129,946bd3,https://imgur.com/a/cmhUqbZ,277,1.533297e+09,"[e3irxuf, e3j7uq9, e3iyuax, e3isp2l, e3jfyse, ...",7.83898,19,F
5,"36F intersex but lean more female, im from the...",7,i2bvwx,https://i.redd.it/3aw7wxtjale51.jpg,24,1.596403e+09,"[g03dcbd, g04fitl, g04h8y2, g04roz0, g04xryv, ...",7.08333,36,F
6,[F18] Let's try this again guys,6,4djlup,http://imgur.com/psZSUFI&Ona3pSG&tbjBZFe&31PgR...,49,1.459935e+09,"[d1rle48, d1rlhhr, d1rwe0v, d1rodi1, d1rktb7, ...",7.42308,18,F
...,...,...,...,...,...,...,...,...,...,...
988,25F re-upload with the username not backward,0,kih9kj,https://imgur.com/a/F40uhIe,16,1.608710e+09,"[ggqt82a, ggqy79h, ggr5te6, ggr3wj0, ggrim3s, ...",2.83333,25,F
989,[22f] I need honestly feedback & rates!!,16,j2urm9,https://i.redd.it/du9wbw5iocq51.png,31,1.601529e+09,"[g77xefu, g7e67rq, g7hn2c0, g7cyx6g, g7c66qi, ...",5.77778,22,F
990,27f new mom. 5'11 170lbs. Rate me.,2,kqgg5h,http://imgur.com/a/mVVJWCN,4,1.609819e+09,"[gi3n6bd, gi5vldf, gi5hpbz, gi5o4xa]",6.25,27,F
991,"M25, would like to know what you think especia...",123,kjm1ua,https://i.redd.it/78f9dse217761.jpg,82,1.608870e+09,"[ggxezy2, ggxskat, ggyyrue, ggxqgcf, ggxwwto, ...",8.28333,25,M


In [35]:
controversial_df[["age"]] = controversial_df[["age"]].apply(pd.to_numeric)
controversial_df[["average_rating"]] = controversial_df[["average_rating"]].apply(pd.to_numeric)
controversial_df[["score"]] = controversial_df[["score"]].apply(pd.to_numeric)

In [36]:
# What's the corelation between attraction and score?
controversial_df['average_rating'].corr(controversial_df['score'])

0.040660561502491074

In [37]:
# How many males are in the controverisial vs females?
controversial_df[controversial_df["gender"] == "M"].shape[0]
# 205 / 452

204

## Scrapping the images of the reddit post, and applying a face detection algorithm for the ethnicity of the user 